## Code setup
Let's start by importing the necessary libraries and setting up the code for the tutorial.

In [3]:
import torch
import torch.nn as nn
import torchvision as tv
import tqdm as tqdm
from lightning import pytorch as pl
from pytorch_lightning.loggers import WandbLogger


In [4]:
class QMNISTClassifier(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        batch_size: int,
        lr: float,
        momentum: float,
        w_decay: float,
        T_max: int,
    ):
        super().__init__()
        self.model = model

        self.criterion = nn.CrossEntropyLoss()

        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        
        train_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_acc", train_acc)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        
        val_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_acc", val_acc)
        
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        
        test_acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_acc", test_acc)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.model.parameters(),
            lr=self.hparams.lr,
            momentum=self.hparams.momentum,
            weight_decay=self.hparams.w_decay,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=self.hparams.T_max
        )
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.QMNIST(
                root="./data",
                train=True,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.hparams.batch_size,
            shuffle=True,
        )
        
    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.QMNIST(
                root="./data",
                train=False,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.hparams.batch_size,
            shuffle=False,
        )
        
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            tv.datasets.QMNIST(
                root="./data",
                train=False,
                download=True,
                transform=tv.transforms.ToTensor(),
            ),
            batch_size=self.hparams.batch_size,
            shuffle=False,
        )


In [5]:
hyperparameters = {
    "batch_size": 128,
    "lr": 0.01,
    "momentum": 0.9,
    "w_decay": 5e-4,
    "T_max": 200,
}


## Model Architectures

In [ ]:
def main(hyperparameters):
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28 * 28, 10),
    )
    
    logger = WandbLogger(
        name="fc",
        project="qmnist",
        save_dir="projects/blog/qmnist/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    mnistTrainer = QMNISTClassifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    
    trainer.fit(mnistTrainer)
    trainer.test(mnistTrainer)

    logger.experiment.finish()

if __name__ == "__main__":
        
    main(hyperparameters)


In [ ]:
def main(hyperparameters):
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28 * 28, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10),
    )
    
    logger = WandbLogger(
        name="mlp",
        project="qmnist",
        save_dir="projects/blog/qmnist/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    mnistTrainer = QMNISTClassifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )
    
    trainer.fit(mnistTrainer)
    trainer.test(mnistTrainer)

    logger.experiment.finish()

if __name__ == "__main__":
        
    main(hyperparameters)


### LeNet-5

![lenet-5](https://www.datasciencecentral.com/wp-content/uploads/2021/10/1lvvWF48t7cyRWqct13eU0w.jpeg)

#### Normal

In [12]:
def main(hyperparameters):
    model = nn.Sequential(
        nn.Conv2d(1, 6, 5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(6, 16, 5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(16 * 5 * 5, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10),
    )

    logger = WandbLogger(
        name="lenet-5",
        project="qmnist",
        save_dir="projects/blog/cifar10/logs",
        log_model=True,
    )
    logger.watch(model, log="all")

    mnistTrainer = QMNISTClassifier(model, **hyperparameters)

    trainer = pl.Trainer(
        max_epochs=20,
        logger=logger,
        val_check_interval=0.5,
        limit_val_batches=0.25,
        enable_progress_bar=False,
    )

    trainer.fit(mnistTrainer)
    trainer.test(mnistTrainer)

    logger.experiment.finish()


if __name__ == "__main__":
    hyperparameters = {
        "batch_size": [8, 16, 32, 64, 128, 256],
        "lr": [0.1, 0.01, 0.001],
        "momentum": [0.9, 0.99],
        "w_decay": [5e-4],
        "T_max": [200],
    }

    for batch_size in hyperparameters["batch_size"]:
        for lr in hyperparameters["lr"]:
            for momentum in hyperparameters["momentum"]:
                for w_decay in hyperparameters["w_decay"]:
                    for T_max in hyperparameters["T_max"]:
                        hyperparameters = {
                            "batch_size": batch_size,
                            "lr": lr,
                            "momentum": momentum,
                            "w_decay": w_decay,
                            "T_max": T_max,
                        }
                        main(hyperparameters)


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 61.7 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
61.7 K    Trainable params
0         Non-trainable params
61.7 K    Total params
0.247     Total estimated model params size (MB)
wandb: ERROR Error while calling W&B API: run qmnist/61ggiv8x not found during createRunFiles (<Response [404]>)
wandb: ERROR Error while calling W&B API: run 61ggiv8x was previously created and deleted; try a new run name (<Response [409]>)
Thread SenderThread:
Traceback (most recent call last):
  File "/Users/karan/Coding/repos/ml-zoo/venv/lib/python3.10/site-packages/wandb/apis/normalize.py", line 41, in wrapper
    return func(*args, **kwargs)
  File "/Users/karan/Coding/repos/ml-zoo/ven

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x28e294be0>> (for post_run_cell), with arguments args (<ExecutionResult object at 29ea16590, execution_count=12 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 29ea16770, raw_cell="def main(hyperparameters):
    model = nn.Sequenti.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/karan/Coding/repos/ml-zoo/projects/blog/qmnist/mnist.ipynb#X13sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe